classifiers: nn.crossentropyloss = -log-likelihood --- use for logp(y) and -logq(y/x) for support <br>
kl-div: <br>
reconstr-loss: set reduction to none and then take mean of losses per image in the total batch. This gives reconstr-loss per image for further computation<br> 

In [1]:
import sys
import random
import numpy as np
import torch
import learn2learn as l2l
import pandas as pd
import os
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from PIL.Image import LANCZOS

In [2]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.9.1101343141/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.9.1101343141/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [3]:
#from data.loaders import Omniglotmix, MiniImageNet
from data.taskers import gen_tasks
from src.zoo.archs import CCVAE, CEncoder, Classifier_VAE, ConvBase
from src.zoo.delpo_utils import setup, inner_adapt_delpo, loss, accuracy
from src.utils2 import Profiler

In [5]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../dataset/mini_imagenet', mode='train', n_ways=5, k_shots=5, q_shots=15, image_transforms=None)

In [5]:
classes = list(range(1623))
random.shuffle(classes)
image_transforms = transforms.Compose([transforms.Resize(28, interpolation=LANCZOS),
                                                    transforms.ToTensor(),
                                                    lambda x: 1.0 - x,
                                                ])
train_tasks = gen_tasks('omniglot', '/home/anuj/Desktop/Work/TU_Delft/research/implement/dataset/omniglot', image_transforms=image_transforms, n_ways=5, k_shots=5, q_shots=15, classes=classes[:1100])

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [106]:
device = 'cuda'
n_ways = 5
k_shots = 1
q_shots = 15
dataset = 'omniglot'
root = '../../dataset/omniglot'
order = False
inner_lr = 0.01
meta_lr = 0.001
reconst_loss = nn.MSELoss(reduction='none')
inner_adapt_steps_train = 5
meta_batch_size = 20

In [107]:
train_tasks, valid_tasks, test_tasks, learner = setup(
    dataset, root, n_ways, k_shots, q_shots, order, inner_lr, device, download='False', task_adapt=False, task_adapt_fn='none', args=args)
opt = optim.Adam(learner.parameters(), meta_lr)

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [16]:
class Args:
    def __init__(self, wt_ce, klwt, rec_wt, beta_l, beta_s):
        #args.wt_ce, args.klwt, args.rec_wt, args.beta_l, args.beta_s
        self.wt_ce = wt_ce
        self.klwt = klwt
        self.rec_wt = rec_wt
        self.beta_l = beta_l
        self.beta_s = beta_s
        self.dataset = 'omniglot'
        self.task_adapt = False

In [17]:
args = Args(10, False, 0.1, 1, 1)

In [108]:
model_path = '../logs/DELPO_omniglot_5-way_1-shot_15-queries/exp1/model_500.pt'
model_name = '../logs/DELPO_omniglot_5-way_1-shot_15-queries/exp1/opt_500.pt'

dict_s = torch.load(model_path).state_dict()
learner.load_state_dict(dict_s)
learner = learner.to('cuda')
opt.load_state_dict(torch.load('./opt.pt'))
learner.train()

MAML(
  (module): CCVAE(
    (encoder): CEncoder(
      (net): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
        (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (8): ReLU()
        (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (11): ReLU()
        (12): Flatten(start_dim=1, end_dim=-1)
      )
      (h1): Linear(in_features=256, out_features=64, bias=True)
      (h2): Linear(in_features=256, o

In [95]:
learner.state_dict()

OrderedDict([('module.encoder.net.0.weight',
              tensor([[[[-0.3042, -0.3240,  0.0983],
                        [ 0.1889,  0.1973, -0.2662],
                        [-0.2694, -0.1842, -0.1137]]],
              
              
                      [[[-0.1531,  0.2689, -0.0521],
                        [-0.1819,  0.0761, -0.0442],
                        [ 0.0893,  0.3041, -0.2767]]],
              
              
                      [[[ 0.0046, -0.2908, -0.0801],
                        [-0.2227, -0.1329,  0.1747],
                        [-0.2689, -0.2431,  0.1932]]],
              
              
                      [[[ 0.3314, -0.0482,  0.2038],
                        [ 0.2654,  0.0167, -0.1711],
                        [ 0.2608,  0.0654, -0.2195]]],
              
              
                      [[[-0.2262,  0.0568, -0.3043],
                        [-0.1639, -0.0998,  0.0619],
                        [-0.2274, -0.1743, -0.2025]]],
              
              


In [103]:
learner.state_dict()

OrderedDict([('module.encoder.net.0.weight',
              tensor([[[[-0.3048, -0.3246,  0.0990],
                        [ 0.1896,  0.1979, -0.2656],
                        [-0.2700, -0.1848, -0.1144]]],
              
              
                      [[[-0.1525,  0.2682, -0.0513],
                        [-0.1824,  0.0755, -0.0448],
                        [ 0.0886,  0.3032, -0.2761]]],
              
              
                      [[[ 0.0053, -0.2915, -0.0807],
                        [-0.2234, -0.1336,  0.1740],
                        [-0.2694, -0.2438,  0.1925]]],
              
              
                      [[[ 0.3307, -0.0475,  0.2044],
                        [ 0.2647,  0.0161, -0.1718],
                        [ 0.2601,  0.0647, -0.2202]]],
              
              
                      [[[-0.2269,  0.0561, -0.3050],
                        [-0.1633, -0.0991,  0.0612],
                        [-0.2267, -0.1750, -0.2032]]],
              
              


In [100]:
ttask = train_tasks.sample()
model = learner.clone()
evaluation_loss, evaluation_accuracy, reconst_img, query_imgs, mu_l, log_var_l, mu_s, log_var_s = inner_adapt_delpo(
                ttask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device, True, args)

In [88]:
evaluation_loss

{'elbo': tensor(34.0709, grad_fn=<AddBackward0>),
 'label_kl': tensor(3.2133, grad_fn=<MeanBackward0>),
 'style_kl': tensor(2.9706, grad_fn=<MeanBackward0>),
 'reconstruction_loss': tensor(115.3331, grad_fn=<MeanBackward0>),
 'classification_loss': tensor(1.6354, grad_fn=<NllLossBackward>)}

In [101]:
evaluation_loss['elbo'].backward()

In [102]:
# Meta backpropagation of gradients
for p in learner.parameters():
    p.grad.data.mul_(1.0 / 5)
opt.step()

In [33]:
tmp = [1, evaluation_accuracy.item()]
tmp = tmp + [a.item() for a in evaluation_loss.values()]
batch_losses.append(tmp)

In [34]:
batch_losses

[[1,
  0.20000000298023224,
  213.8948516845703,
  15.829955101013184,
  14.429597854614258,
  2242.850830078125,
  1.6120678186416626]]

In [9]:
learner = torch.load('./model.pt')


In [10]:
learner.to(device)

MAML(
  (module): CCVAE(
    (encoder): CEncoder(
      (net): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
        (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (8): ReLU()
        (9): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (11): ReLU()
        (12): Flatten(start_dim=1, end_dim=-1)
      )
      (h1): Linear(in_features=256, out_features=64, bias=True)
      (h2): Linear(in_features=256, o

In [11]:
opt.load_state_dict(torch.load('./opt.pt'))

In [12]:
## Training ##
for iter in range(2):
    opt.zero_grad()
    batch_losses = []

    for batch in range(meta_batch_size):
        ttask = train_tasks.sample()
        model = learner.clone()
        if (batch == 0):
            evaluation_loss, evaluation_accuracy, reconst_img, query_imgs, mu_l, log_var_l, mu_s, log_var_s = inner_adapt_delpo(
                ttask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device, True, args)
            
            # Logging train-task images and latents
            # di = {"reconst_examples": reconst_img.detach().to('cpu'), "gt_examples": query_imgs.detach().to('cpu')}
            # dl = {"label_latents": [mu_l.detach().to('cpu'), log_var_l.detach().to('cpu')],
            #       "style_latents": [mu_s.detach().to('cpu'), log_var_s.detach().to('cpu')]}
            # profiler.log_data(di, iter, 'images', 'train')
            # profiler.log_data(dl, iter, 'latents', 'train')

        else:
            evaluation_loss, evaluation_accuracy = inner_adapt_delpo(
                ttask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device, False, args)

        evaluation_loss['elbo'].backward()

        # Logging per train-task losses and accuracies
        # tmp = [(iter*meta_batch_size)+batch, evaluation_accuracy.item()]
        # tmp = tmp + [a.item() for a in evaluation_loss.values()]
        # batch_losses.append(tmp)

    #     wandb.log(dict({f"train/{key}": loss.item() for _, (key, loss) in enumerate(evaluation_loss.items())},
    #               **{'train/accuracies': evaluation_accuracy.item(), 'train/task': (iter*args.meta_batch_size)+batch}))

    # rimages = wandb.Image(reconst_img, caption="Reconstructed Query Images")
    # qimages = wandb.Image(query_imgs, caption="Query Images")
    # wandb.log({"reconst_examples": rimages, "gt_examples": qimages})

    vtask = valid_tasks.sample()
    model = learner.clone()
    if iter % 2 == 0:
        validation_loss, validation_accuracy, reconst_img, query_imgs, mu_l, log_var_l, mu_s, log_var_s = inner_adapt_delpo(
            vtask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device, True, args)
        # # Logging valid-task images and latents
        # di = {"reconst_examples": reconst_img.to('cpu'), "gt_examples": query_imgs.to('cpu')}
        # dl = {"label_latents": [mu_l.to('cpu'), log_var_l.to('cpu')],
        #     "style_latents": [mu_s.to('cpu'), log_var_s.to('cpu')]}
        # profiler.log_data(di, iter, 'images', 'valid')
        # profiler.log_data(dl, iter, 'latents', 'valid')

    else:
        validation_loss, validation_accuracy = inner_adapt_delpo(
            vtask, reconst_loss, model, n_ways, k_shots, q_shots, inner_adapt_steps_train, device, False, args)

    # Logging per validation-task losses and accuracies
    # tmp = [iter, validation_accuracy.item()]
    # tmp = tmp + [a.item() for a in validation_loss.values()]

    # wandb.log(dict({f"valid/{key}": loss.item() for _, (key, loss) in enumerate(validation_loss.items())},
    #           **{'valid/accuracies': validation_accuracy.item(), 'valid/task': iter}))
# Meta backpropagation of gradients
    for p in learner.parameters():
        p.grad.data.mul_(1.0 / 5)
    opt.step()

    # Saving the Logs
    # profiler.log_csv(batch_losses, 'train')
    # profiler.log_csv(tmp, 'valid')

#torch.save(learner, f='../repro')


In [72]:
torch.save(learner.to('cpu').state_dict(), './model.pt')
torch.save(opt.state_dict(), './opt.pt')    

In [13]:
opt.state_dict()

{'state': {0: {'step': 2,
   'exp_avg': tensor([[[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
   

In [14]:
opt.state_dict()

{'state': {0: {'step': 2,
   'exp_avg': tensor([[[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
             [nan, nan, nan]]],
   
   
           [[[nan, nan, nan],
             [nan, nan, nan],
   

In [20]:
log_var_l.shape

torch.Size([2, 64])

In [16]:
for i in range(2,5):
    print('hi')

hi
hi
hi


In [20]:
test_tasks.sample

[tensor([[[[179., 164., 166.,  ..., 219., 225., 229.],
           [162., 171., 162.,  ..., 212., 220., 218.],
           [154., 151., 161.,  ..., 216., 215., 215.],
           ...,
           [255., 255., 255.,  ..., 255., 255., 255.],
           [255., 255., 255.,  ..., 255., 255., 255.],
           [255., 255., 255.,  ..., 255., 255., 255.]],
 
          [[171., 160., 163.,  ..., 223., 232., 233.],
           [156., 166., 158.,  ..., 212., 230., 229.],
           [149., 144., 154.,  ..., 216., 227., 227.],
           ...,
           [255., 255., 255.,  ..., 255., 255., 255.],
           [255., 255., 255.,  ..., 255., 255., 255.],
           [255., 255., 255.,  ..., 255., 255., 255.]],
 
          [[157., 142., 155.,  ..., 235., 239., 243.],
           [137., 147., 146.,  ..., 220., 235., 238.],
           [128., 126., 135.,  ..., 225., 233., 238.],
           ...,
           [255., 255., 255.,  ..., 255., 255., 255.],
           [255., 255., 255.,  ..., 255., 255., 255.],
           

In [20]:
torch.save(learner, './model.pt')

In [22]:
learner = torch.load('./model.pt')

In [25]:
learner = learner.to('cpu')

In [27]:
learner(vtask[0])

(tensor([[[[0.5460, 0.7005, 0.7540,  ..., 0.6607, 0.5247, 0.6521],
           [0.3869, 0.4778, 0.6000,  ..., 0.5082, 0.4487, 0.5776],
           [0.5202, 0.7316, 0.7623,  ..., 0.5487, 0.4707, 0.5494],
           ...,
           [0.4386, 0.4401, 0.5506,  ..., 0.5411, 0.4927, 0.5433],
           [0.4770, 0.3676, 0.4371,  ..., 0.3584, 0.3767, 0.4090],
           [0.4787, 0.3858, 0.3679,  ..., 0.3389, 0.3394, 0.4149]],
 
          [[0.4559, 0.4518, 0.4888,  ..., 0.7013, 0.6391, 0.3140],
           [0.3996, 0.3678, 0.3866,  ..., 0.4096, 0.3197, 0.2149],
           [0.4501, 0.2640, 0.2461,  ..., 0.2523, 0.1991, 0.1818],
           ...,
           [0.2929, 0.2676, 0.3173,  ..., 0.3396, 0.2282, 0.2554],
           [0.3211, 0.2309, 0.2781,  ..., 0.4156, 0.2826, 0.2474],
           [0.5293, 0.4962, 0.4777,  ..., 0.4851, 0.3834, 0.4208]],
 
          [[0.7325, 0.5827, 0.5901,  ..., 0.7413, 0.6871, 0.7642],
           [0.7580, 0.6519, 0.6883,  ..., 0.7370, 0.6829, 0.7204],
           [0.7148, 0.49

In [11]:

## Testing ##
print('Testing on held out classes')

for i, tetask in enumerate(test_tasks):
    # wandb.define_metric("accuracies", summary="max")
    # wandb.define_metric("accuracies", summary="mean")

    model = learner.clone()
    #tetask = test_tasks.sample()
    evaluation_loss, evaluation_accuracy = inner_adapt_delpo(
        tetask, reconst_loss, model, n_ways, k_shots, q_shots, 5, device, False, args)

    # Logging per test-task losses and accuracies
    # tmp = [iter, evaluation_accuracy.item()]
    # tmp.append(a.item() for a in evaluation_loss.values())
    # profiler.log_csv(tmp, 'test')
    # # wandb.log(dict({f"test/{key}": loss.item() for _, (key, loss) in enumerate(evaluation_loss.items())},
    #             **{'test/accuracies': evaluation_accuracy.item(), 'test/task': i}))


Testing on held out classes


KeyboardInterrupt: 